EXP-07 (AGGLOMERITIVE ALGORITM)

In [ ]:
#Step 1- Read csv file with n features
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage




# Define the points

"""data = {
    'Point': ['P1', 'P2', 'P3', 'P4', 'P5'],
    'X': [4, 8, 15, 24, 24],
    'Y': [4, 4, 8, 4, 12]
}

df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)
print(df)"""


data = pd.read_csv('agglo_data.csv')
coordinates = data[['X', 'Y']].values
print(data)

#Step2- Find distance between object with every
#other object. You can use Euclidean function
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

#Step 3 - Find Distance Matrix using Euclidean distance formula
# Initialize a distance matrix
num_points = len(coordinates)
distance_matrix = np.zeros((num_points, num_points))

# Compute the distance matrix
for i in range(num_points):
    for j in range(num_points):
        if i != j:
            distance_matrix[i, j] = euclidean_distance(coordinates[i], coordinates[j])

# Display the distance matrix
distance_df = pd.DataFrame(distance_matrix, columns=data['Point'], index=data['Point'])
print("\nDistance Matrix:\n", distance_df)

# Step 4: Find Minimum Distance and Involved Objects for All Points
min_distance = float('inf')
point_pair = (None, None)

for i in range(num_points):
    for j in range(num_points):
        if i != j and distance_matrix[i, j] < min_distance:
            min_distance = distance_matrix[i, j]
            point_pair = (data['Point'][i], data['Point'][j])

# Display the results
print(f"Minimum distance for all points: {point_pair[0]} and {point_pair[1]} – minimum distance = {min_distance:.2f}")

#Step 5 . Merge these objects P1and P2. Now (P1,P2) is one unit/cluster.
# Initialize clusters
clusters = [[data['Point'][i]] for i in range(len(data))]
print("Initial Clusters:", clusters)
# Merge P1 and P2

if point_pair[0] and point_pair[1]:
    merged_cluster = [point_pair[0], point_pair[1]]

    # Update the clusters list
    clusters = [c for c in clusters if c[0] not in merged_cluster]  # Remove old clusters
    clusters.append(merged_cluster)  # Add the new cluster

print("Updated Clusters after merging:", clusters)

#Step 6 : Recalculate the distances.

def update_cluster_coordinates(clusters, original_coordinates):
    new_coordinates = []
    for cluster in clusters:
        if len(cluster) == 1:
            new_coordinates.append(original_coordinates[data['Point'].tolist().index(cluster[0])])
        else:
            avg_coordinates = np.mean([original_coordinates[data['Point'].tolist().index(point)] for point in cluster], axis=0)
            new_coordinates.append(avg_coordinates)

    return np.array(new_coordinates)

# Calculate new cluster coordinates
updated_coordinates = update_cluster_coordinates(clusters, coordinates)

# Initialize the new distance matrix
num_updated_points = len(updated_coordinates)
new_distance_matrix = np.zeros((num_updated_points, num_updated_points))

# Compute the new distance matrix
for i in range(num_updated_points):
    for j in range(num_updated_points):
        if i != j:
            new_distance_matrix[i, j] = euclidean_distance(updated_coordinates[i], updated_coordinates[j])

# Create a DataFrame for easier reading
new_distance_df = pd.DataFrame(new_distance_matrix, columns=[str(cluster) for cluster in clusters], index=[str(cluster) for cluster in clusters])
print("New Distance Matrix:\n", new_distance_df)



# Perform hierarchical clustering using Ward's method
Z = linkage(df[['X', 'Y']], method='ward')

# Plot the dendrogram
plt.figure(figsize=(8, 6))
dendrogram(Z, labels=df['Point'].values)
plt.title('Dendrogram', fontsize=18, color='red')
plt.xlabel('Points')
plt.ylabel('Distance')
plt.show()
